In [1]:
!pip install transformers datasets pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00


In [2]:
from huggingface_hub import notebook_login

notebook_login()


In [4]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import pyarrow.parquet as pq

In [5]:
!wget "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_Input.zip"
!wget "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_Input.zip"
!wget "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Test_Input.zip"

--2024-01-23 04:53:00--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_Input.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 52.216.62.121, 52.217.105.228, 52.217.113.105, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)|52.216.62.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2771732744 (2.6G) [application/zip]
Saving to: ‘ISIC2018_Task3_Training_Input.zip’

ISIC2018_Task3_Trai 100%[===================>]   2.58G  59.1MB/s    in 43s     

2024-01-23 04:53:43 (61.7 MB/s) - ‘ISIC2018_Task3_Training_Input.zip’ saved [2771732744/2771732744]

--2024-01-23 04:53:43--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_Input.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 3.5.8.185, 3.5.28.234, 3.5.29.174, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challenge-d

In [6]:
!unzip /content/ISIC2018_Task3_Test_Input.zip
!unzip /content/ISIC2018_Task3_Training_Input.zip
!unzip /content/ISIC2018_Task3_Validation_Input.zip

Streaming output truncated to the last 5000 lines.
 extracting: ISIC2018_Task3_Training_Input/ISIC_0029519.jpg  
 extracting: ISIC2018_Task3_Training_Input/ISIC_0029520.jpg  
  inflating: ISIC2018_Task3_Training_Input/ISIC_0029521.jpg  
 extracting: ISIC2018_Task3_Training_Input/ISIC_0029522.jpg  
  inflating: ISIC2018_Task3_Training_Input/ISIC_0029523.jpg  
  inflating: ISIC2018_Task3_Training_Input/ISIC_0029524.jpg  
  inflating: ISIC2018_Task3_Training_Input/ISIC_0029525.jpg  
 extracting: ISIC2018_Task3_Training_Input/ISIC_0029526.jpg  
 extracting: ISIC2018_Task3_Training_Input/ISIC_0029527.jpg  
  inflating: ISIC2018_Task3_Training_Input/ISIC_0029528.jpg  
  inflating: ISIC2018_Task3_Training_Input/ISIC_0029529.jpg  
 extracting: ISIC2018_Task3_Training_Input/ISIC_0029530.jpg  
 extracting: ISIC2018_Task3_Training_Input/ISIC_0029531.jpg  
 extracting: ISIC2018_Task3_Training_Input/ISIC_0029532.jpg  
  inflating: ISIC2018_Task3_Training_Input/ISIC_0029533.jpg  
 extracting: ISIC20

In [7]:
train_dir = "/content/ISIC2018_Task3_Training_Input"
val_dir = "/content/ISIC2018_Task3_Validation_Input.zip"
test_dir = "/content/ISIC2018_Task3_Test_Input"

In [8]:
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_GroundTruth.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_GroundTruth.zip
!wget https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Test_GroundTruth.zip


--2024-01-23 04:54:30--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Training_GroundTruth.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 3.5.8.150, 52.217.174.209, 52.216.12.12, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)|3.5.8.150|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36681 (36K) [application/zip]
Saving to: ‘ISIC2018_Task3_Training_GroundTruth.zip’

ISIC2018_Task3_Trai 100%[===================>]  35.82K  --.-KB/s    in 0.03s   

2024-01-23 04:54:30 (1.25 MB/s) - ‘ISIC2018_Task3_Training_GroundTruth.zip’ saved [36681/36681]

--2024-01-23 04:54:30--  https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task3_Validation_GroundTruth.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 3.5.8.150, 52.217.174.209, 52.216.12.12, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challen

In [9]:
!unzip /content/ISIC2018_Task3_Test_GroundTruth.zip
!unzip  /content/ISIC2018_Task3_Training_GroundTruth.zip
!unzip /content/ISIC2018_Task3_Validation_GroundTruth.zip

Archive:  /content/ISIC2018_Task3_Test_GroundTruth.zip
   creating: ISIC2018_Task3_Test_GroundTruth/
  inflating: ISIC2018_Task3_Test_GroundTruth/LICENSE.txt  
  inflating: ISIC2018_Task3_Test_GroundTruth/ATTRIBUTION.txt  
  inflating: ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth.csv  
Archive:  /content/ISIC2018_Task3_Training_GroundTruth.zip
   creating: ISIC2018_Task3_Training_GroundTruth/
  inflating: ISIC2018_Task3_Training_GroundTruth/ATTRIBUTION.txt  
  inflating: ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv  
  inflating: ISIC2018_Task3_Training_GroundTruth/LICENSE.txt  
Archive:  /content/ISIC2018_Task3_Validation_GroundTruth.zip
   creating: ISIC2018_Task3_Validation_GroundTruth/
  inflating: ISIC2018_Task3_Validation_GroundTruth/ATTRIBUTION.txt  
  inflating: ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth.csv  
  inflating: ISIC2018_Task3_Validation_GroundTruth/LICENSE.txt  


In [91]:
!rm /content/ISIC2018_Task3_Test_GroundTruth.zip
!rm  /content/ISIC2018_Task3_Training_GroundTruth.zip
!rm /content/ISIC2018_Task3_Validation_GroundTruth.zip

!rm /content/ISIC2018_Task3_Training_Input.zip
!rm /content/ISIC2018_Task3_Test_Input.zip
!rm /content/ISIC2018_Task3_Validation_Input.zip


rm: cannot remove '/content/ISIC2018_Task3_Test_GroundTruth.zip': No such file or directory
rm: cannot remove '/content/ISIC2018_Task3_Training_GroundTruth.zip': No such file or directory
rm: cannot remove '/content/ISIC2018_Task3_Validation_GroundTruth.zip': No such file or directory


In [11]:
test_df = pd.read_csv("/content/ISIC2018_Task3_Test_GroundTruth/ISIC2018_Task3_Test_GroundTruth.csv")
train_df = pd.read_csv("/content/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_GroundTruth.csv")
val_df = pd.read_csv("/content/ISIC2018_Task3_Validation_GroundTruth/ISIC2018_Task3_Validation_GroundTruth.csv")

### Create HF dataset

In [86]:
def create_dataset(df, directory):

  classes = df.columns.remove()
  labels = []
  for index, row in df.iterrows():
    # Finding column name with value 1
    label = row[row == 1].index.tolist()

    # Appending to the list
    labels.append(label[0])
    img_file = os.path.join(directory, row[0], ".jpg")
    images.append(img_file)

  ds = datasets.Dataset.from_dict({"image": images, "label": })




In [87]:
  create_dataset(train_df)

['NV',
 'NV',
 'NV',
 'NV',
 'MEL',
 'NV',
 'BKL',
 'MEL',
 'NV',
 'MEL',
 'NV',
 'NV',
 'DF',
 'NV',
 'NV',
 'NV',
 'NV',
 'MEL',
 'BKL',
 'NV',
 'NV',
 'NV',
 'NV',
 'AKIEC',
 'DF',
 'BCC',
 'BCC',
 'MEL',
 'NV',
 'NV',
 'BKL',
 'BKL',
 'BKL',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'BCC',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'MEL',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'BKL',
 'NV',
 'BCC',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'MEL',
 'NV',
 'NV',
 'VASC',
 'BKL',
 'AKIEC',
 'NV',
 'NV',
 'VASC',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'BKL',
 'BKL',
 'BKL',
 'NV',
 'NV',
 'DF',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'DF',
 'NV',
 'NV',
 'NV',
 'MEL',
 'NV',
 'VASC',
 'BCC',
 'NV',
 'NV',
 'NV',
 'NV',
 'BKL',
 'BKL',
 'MEL',
 'BCC',
 'BKL',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV',
 'AKIEC',
 'NV',
 'BKL',
 'NV',
 'BKL',
 'NV',
 'NV',
 'NV',
 'BKL',
 'NV',
 'NV',
 'NV',
 'NV',
 'BCC',
 'BCC',
 'NV',
 'NV',
 'BKL',
 'BCC',
 'NV',
 'NV',
 'NV',
 'NV',
 'NV'

In [ ]:
train_df.columns

ISIC_0024306
ISIC_0024307
ISIC_0024308
ISIC_0024309
ISIC_0024310
ISIC_0024311
ISIC_0024312
ISIC_0024313
ISIC_0024314
ISIC_0024315
ISIC_0024316
ISIC_0024317
ISIC_0024318
ISIC_0024319
ISIC_0024320
ISIC_0024321
ISIC_0024322
ISIC_0024323
ISIC_0024324
ISIC_0024325
ISIC_0024326
ISIC_0024327
ISIC_0024328
ISIC_0024329
ISIC_0024330
ISIC_0024331
ISIC_0024332
ISIC_0024333
ISIC_0024334
ISIC_0024335
ISIC_0024336
ISIC_0024337
ISIC_0024338
ISIC_0024339
ISIC_0024340
ISIC_0024341
ISIC_0024342
ISIC_0024343
ISIC_0024344
ISIC_0024345
ISIC_0024346
ISIC_0024347
ISIC_0024348
ISIC_0024349
ISIC_0024350
ISIC_0024351
ISIC_0024352
ISIC_0024353
ISIC_0024354
ISIC_0024355
ISIC_0024356
ISIC_0024357
ISIC_0024358
ISIC_0024359
ISIC_0024360
ISIC_0024361
ISIC_0024362
ISIC_0024363
ISIC_0024364
ISIC_0024365
ISIC_0024366
ISIC_0024367
ISIC_0024368
ISIC_0024369
ISIC_0024370
ISIC_0024371
ISIC_0024372
ISIC_0024373
ISIC_0024374
ISIC_0024375
ISIC_0024376
ISIC_0024377
ISIC_0024378
ISIC_0024379
ISIC_0024380
ISIC_0024381
ISIC_0024382

## Create a column image that contains the binary of all the images

In [12]:
import os
import datasets

def create_image_folder_dataset(root_path):
  """creates `Dataset` from image folder structure"""


